In [2]:
import pandas as pd
import pymysql
import re
from ingredient_text_utils import unit_keywords



# ✅ 외부 모듈에서 숫자 단위 매칭용 normalize 함수 import
from ingredient_text_utils import normalize_quantity_expression


# ✅ 사전 불러오기
ingredient_df = pd.read_csv(r"C:\Users\user\Desktop\RefrigeratorCode\ingredient_profile_dict_v1.csv", encoding="cp949")

# ✅ "대분류"가 '재료','포장/제품'인 항목만 필터링
ingredient_df_filtered = ingredient_df[ingredient_df["대분류"].isin(["재료", "포장/제품"])]


# ✅ 사전 분리: 일반 / 한 글자
normal_dict, short_dict = {}, {}

for _, row in ingredient_df_filtered.iterrows():
    if pd.isna(row["ingredient_name"]):
        continue
    base_name = str(row["ingredient_name"]).strip()
    synonyms = str(row["synonyms"]).split(", ") if not pd.isna(row["synonyms"]) else []
    names = [base_name] + synonyms

    for name in names:
        name = name.strip()
        if not name:
            continue
        if len(name) == 1:
            short_dict[name] = base_name
        else:
            normal_dict[name] = base_name
            
# ✅ 한 글자 재료는 "단독 단어" + "짧은 문장"에서만 인정
def is_valid_short_match(word, line):
    pattern = rf"(?<![가-힣A-Za-z]){re.escape(word)}(?![가-힣A-Za-z])"
    return bool(re.search(pattern, line)) and len(line.strip()) <= 25

# ✅ DB 연결
db = pymysql.connect(
    host='localhost',
    user='root',
    password='sk784512!!',
    db='refrigerator',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)
cursor = db.cursor()

# ✅ 레시피 불러오기
cursor.execute("SELECT id, content FROM recipes")
recipes = cursor.fetchall()

# ✅ 의미 키워드 줄 정제 함수
def clean_meaning_line(line):
    # 0. 앞쪽 장식 기호 제거 (기존보다 범위 확대)
    line = re.sub(r"^[*#■▶※•★●▷➤➡️\-\s]+", "", line)

    # 1. 괄호 안 내용 제거
    line = re.sub(r"\([^)]*\)", "", line)

    # 2. 껍데기 괄호 제거
    line = re.sub(r"[{}\[\]<>]", "", line)

    # 3. 뒤쪽 장식 기호도 제거 (기존에서 * 추가)
    line = re.sub(r"[:~•\-\.\*\s]+$", "", line.strip())

    return line.strip()



# ✅ used_ingredients_block 추출 함수 + 이유 반환
def extract_best_ingredient_block(text):
    normalized_text = normalize_quantity_expression(text)  # ✅ 여기 한 줄 추가
    lines = normalized_text.split("\n")  # ✅ 정규화된 텍스트 기준으로 줄 분리
    num_lines = len(lines)
    meaning_keywords = [
        "재료", "준비물", "준비할 재료", "준비할재료", "준비하실재료", "준비하실 재료",
        "필요해요", "필요 해요", "재료 안내", "재료안내", "주재료", "주 재료",
        "재료는요", "준비하이소", "요리재료", "요리 재료", "재료 준비", "재료준비"
    ]
    all_keys = list(normal_dict.keys()) + list(short_dict.keys())

    # ✅ 단위 키워드 정규식 생성 함수 (특정 줄에서 단위 키워드 패턴이 있는지 감지하는 역할)
    def strict_quantity_expression(unit):
        korean_numerals = ['한', '두', '세', '네', '다섯', '여섯', '일곱', '여덟', '아홉', '열']
        arabic_pattern = r"(?:[0-9]+|[일이삼사오육칠팔구십백천만]+)"
        korean_pattern = r"(?:{})".format("|".join(korean_numerals))
        combined_pattern = rf"(?:{arabic_pattern}|{korean_pattern})\s*{re.escape(unit)}(?![가-힣])"
        return re.compile(combined_pattern)

    # ✅ 단위 키워드 패턴 컴파일
    compiled_unit_patterns = {
        unit: strict_quantity_expression(unit) for unit in unit_keywords
    }




    # ✅ 블록 확장 함수: 이후 3줄 동안 단위 키워드가 1회라도 나오면 계속 확장 반복
    def extend_block_dynamically(start_idx, base_length):
        end_idx = min(len(lines), start_idx + base_length)
        last_unit_line = end_idx - 1  # 기본 블록 끝
        i = end_idx

        buffer = []  # 최근 3줄 저장
        while i < len(lines):
            buffer.append(lines[i])
            if len(buffer) > 3:
                buffer.pop(0)

            # 최근 3줄 중 하나라도 단위 키워드 포함 시 계속 확장
            match_found = any(
                any(pattern.search(buf_line) for pattern in compiled_unit_patterns.values())
                for buf_line in buffer
            )

            if match_found:
                last_unit_line = i  # 마지막 감지 위치 갱신
                i += 1
            else:
                break

        # 마지막 감지된 줄 +3줄 포함
        final_end = last_unit_line + 1
        return "\n".join(lines[start_idx:final_end])




    # 1️⃣ 의미 기반 키워드 탐색
    for i, line in enumerate(lines):
        line_clean = clean_meaning_line(line.strip())
        for kw in meaning_keywords:
            if line_clean == kw:
                block = extend_block_dynamically(start_idx=max(0, i - 3), base_length=20)  # ← 여기 숫자만 늘려줘
                reason = f"의미 기반 키워드 탐색 (정제 후: {kw})"
                return block, reason


    # 2️⃣ 계량 단위 기반 시작점 탐색
    for i, line in enumerate(lines):
        matched_units = []
        for unit, pattern in compiled_unit_patterns.items():
            match = pattern.search(line)
            if match:
                pre_unit_text = line[:match.start()]
                if any(re.search(rf"{re.escape(ingredient)}\s*$", pre_unit_text) for ingredient in normal_dict):
                    matched_units.append(unit)
        if matched_units:
            block = extend_block_dynamically(start_idx=max(0, i - 3), base_length=15)
            reason = f"단위 키워드 기반 시작점 탐색 ({', '.join(matched_units)})"
            return block, reason

    # 3️⃣ 밀도 기반 탐색 (fallback)
    best_score, best_start = 0, 0
    for i in range(0, max(1, num_lines - 4)):
        window = "\n".join(lines[i:i + 5])
        score = sum(1 for word in all_keys if word in window)
        if score > best_score:
            best_score = score
            best_start = i
    block = extend_block_dynamically(start_idx=max(0, best_start - 3), base_length=8)
    reason = "밀도 기반 Fallback"
    return block, reason









# ✅ 재료 추출 함수
def extract_ingredients_from_block(block):
    found = set()
    matched_spans = []

    for word in sorted(normal_dict, key=lambda x: -len(x)):
        for match in re.finditer(re.escape(word), block):
            span = match.span()
            if any(start < span[1] and span[0] < end for (start, end) in matched_spans):
                continue
            matched_spans.append(span)
            found.add(normal_dict[word])
            break

    for word in short_dict:
        if is_valid_short_match(word, block):
            found.add(short_dict[word])

    return list(found)

# ✅ DB 업데이트
total = len(recipes)
for idx, row in enumerate(recipes, 1):
    rid = row["id"]
    content = row["content"]
    block, reason = extract_best_ingredient_block(content)
    used = extract_ingredients_from_block(block)
    used_joined = ", ".join(sorted(set(used)))

    cursor.execute(
        "UPDATE recipes SET used_ingredients = %s, used_ingredients_block = %s, block_reason = %s WHERE id = %s",
        (used_joined, block, reason, rid)
    )

    if idx % 10 == 0 or idx == total:
        percent = round(idx / total * 100)
        print(f"🔄 진행 중: {idx}/{total} ({percent}%) 완료", flush=True)

# ✅ 종료
db.commit()
cursor.close()
db.close()
print("✅ used_ingredients, used_ingredients_block, block_reason 업데이트 완료!")

🔄 진행 중: 10/794 (1%) 완료
🔄 진행 중: 20/794 (3%) 완료
🔄 진행 중: 30/794 (4%) 완료
🔄 진행 중: 40/794 (5%) 완료
🔄 진행 중: 50/794 (6%) 완료
🔄 진행 중: 60/794 (8%) 완료
🔄 진행 중: 70/794 (9%) 완료
🔄 진행 중: 80/794 (10%) 완료
🔄 진행 중: 90/794 (11%) 완료
🔄 진행 중: 100/794 (13%) 완료
🔄 진행 중: 110/794 (14%) 완료
🔄 진행 중: 120/794 (15%) 완료
🔄 진행 중: 130/794 (16%) 완료
🔄 진행 중: 140/794 (18%) 완료
🔄 진행 중: 150/794 (19%) 완료
🔄 진행 중: 160/794 (20%) 완료
🔄 진행 중: 170/794 (21%) 완료
🔄 진행 중: 180/794 (23%) 완료
🔄 진행 중: 190/794 (24%) 완료
🔄 진행 중: 200/794 (25%) 완료
🔄 진행 중: 210/794 (26%) 완료
🔄 진행 중: 220/794 (28%) 완료
🔄 진행 중: 230/794 (29%) 완료
🔄 진행 중: 240/794 (30%) 완료
🔄 진행 중: 250/794 (31%) 완료
🔄 진행 중: 260/794 (33%) 완료
🔄 진행 중: 270/794 (34%) 완료
🔄 진행 중: 280/794 (35%) 완료
🔄 진행 중: 290/794 (37%) 완료
🔄 진행 중: 300/794 (38%) 완료
🔄 진행 중: 310/794 (39%) 완료
🔄 진행 중: 320/794 (40%) 완료
🔄 진행 중: 330/794 (42%) 완료
🔄 진행 중: 340/794 (43%) 완료
🔄 진행 중: 350/794 (44%) 완료
🔄 진행 중: 360/794 (45%) 완료
🔄 진행 중: 370/794 (47%) 완료
🔄 진행 중: 380/794 (48%) 완료
🔄 진행 중: 390/794 (49%) 완료
🔄 진행 중: 400/794 (50%) 완료
🔄 진행 중: 410/794 